### https://machinelearningmastery.com/building-a-regression-model-in-pytorch/

In [1]:
import os, datetime
import pandas as pd
import numpy as np
import seaborn as sn
import copy
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader

from sklearn import tree, svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = .5
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (16, 10)

In [2]:
np.set_printoptions(precision=4, suppress=True)

In [3]:
# %matplotlib inline
%matplotlib qt

In [39]:
# SHFE.rb2305.2023-04-18.30503.h5
# SHFE.rb2305.2022-11-09.32951.h5
# SHFE.rb2307.2023-05-16.40754.h5
# SHFE.rb2305.2022-11-09.32951.h5

store = pd.HDFStore(os.path.join(SAVE_PATH, 'SHFE.rb.2021-08-31.41391.h5'))
df_x = store['X']
df_y = store['y']
last_price = store['last_price']
store.close()

In [40]:
X = normalize(df_x.values)
y = df_y.values

In [41]:
X_test = torch.tensor(X, dtype=torch.float32, device=0)
y_test = torch.tensor(y, dtype=torch.float32, device=0).reshape(-1, 1)

In [42]:
class FutureTrendModule(nn.Module):
    def __init__(self):
        super(FutureTrendModule, self).__init__()
        
        # input batch_size 300
        self.fc1 = nn.Sequential(
            nn.Linear(900, 128),
            nn.ReLU()
        )
        
        self.lstm = nn.Sequential(
            nn.LSTM(input_size=128, hidden_size=64, num_layers=3, batch_first=True)
        )
        
        self.dropout = nn.Dropout(0.2)
        
        # input batch_size 64
        self.output_fc = nn.Linear(64, 1)

    def forward(self, x):
        fc1 = self.fc1(x)
        lstm1, (hn, cn) = self.lstm(fc1)
        dropout = self.dropout(lstm1)
        outs = self.output_fc(dropout)
        return outs, fc1, lstm1, hn, dropout
    

In [43]:
# class FutureTrendModule(nn.Module):
#     def __init__(self):
#         super(FutureTrendModule, self).__init__()
        
#         # input batch_size 300
#         self.fc1 = nn.Sequential(
#             nn.Linear(900, 128),
#         )
        
#         # input batch_size 1 128
#         self.conv1 = nn.Sequential(
#             nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1),
#             nn.ReLU(True),
# #             nn.AvgPool1d(3, 2),
#             nn.MaxPool1d(3, 2),
#         )
        
#         # input batch_size 64 64
#         self.conv2 = nn.Sequential(
#             nn.Conv1d(64, 32, kernel_size=3, stride=1, padding=2),
#             nn.ReLU(True),
# #             nn.AvgPool1d(3, 2),
#             nn.MaxPool1d(3, 2),
#         )
        
#         # input batch_size 32 32
#         self.conv3 = nn.Sequential(
#             nn.Conv1d(32, 16, kernel_size=3, stride=1, padding=2),
#             nn.ReLU(True),
# #             nn.AvgPool1d(3, 2),
#             nn.MaxPool1d(3, 2),
#         )
        
#         # input batch_size 256
#         self.output_fc1 = nn.Linear(256, 128)
#         # input batch_size 128
#         self.output_fc2 = nn.Linear(128, 1)

#     def forward(self, x):
# #         return self.model(x)
#         fc1 = self.fc1(x)
#         fc1 = fc1.view(-1, 1, 128)
#         conv1 = self.conv1(fc1)
#         conv2 = self.conv2(conv1)
#         conv3 = self.conv3(conv2)
#         outs = conv3.view(-1, 256)
#         out_fc1 = self.output_fc1(outs)
#         outs = self.output_fc2(out_fc1)
#         return outs, fc1, conv1, conv2, conv3, out_fc1
    

In [44]:
model = FutureTrendModule().to(0)
model.load_state_dict(torch.load('D:/option/models/model_0608_01.pt'))

<All keys matched successfully>

In [45]:
model.eval()
# y_pred, fc1, conv1, conv2, conv3, out_fc1 = model(X_test)
y_pred, fc1, lstm1, dropout = model(X_test)

In [27]:
y_pred.mean()

tensor(-0.0002, device='cuda:0', grad_fn=<MeanBackward0>)

In [28]:
y_test.mean()

tensor(3.3324e-05, device='cuda:0')

In [46]:
plt.plot(y_pred.cpu().detach().numpy().reshape(-1), label='pred')
plt.plot(df_y.values.reshape(-1), label='real')
plt.legend()
plt.show()

In [48]:
fig, ax1 = plt.subplots()
ax1.plot(y_pred.cpu().detach().numpy().reshape(-1), label='pred')
ax1.plot(df_y.values.reshape(-1), label='real')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(last_price, label='tick', c='y')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.legend()
plt.show()

In [26]:
plt.plot(last_price, label='tick')
plt.legend()
plt.show()

In [31]:
sn.displot(y_test.cpu().detach().numpy().reshape(-1), kde=True)

In [32]:
sn.displot(y_pred.cpu().detach().numpy().reshape(-1), kde=True)

In [95]:
last_price

array([-1.2189, -1.2278, -1.2367, ...,  0.1211,  0.1156,  0.11  ])

In [54]:
df.loc[check_date].last_price.shape

(40655,)

In [67]:
last_price.shape

(15908, 300)